In [3]:
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import numpy as np 
from scipy.sparse import csr_matrix

In [4]:
reviews = pd.read_csv("reviews.csv")

In [5]:
reviews

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149
...,...,...,...,...,...,...,...
192107,240067,Unicorn819,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '5', 'Animation': '1...",https://myanimelist.net/reviews.php?id=240067
192108,285777,ShizzoSVH,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=285777
192109,286904,AlluMan96,1281,\n \n \n \n ...,3,"{'Overall': '3', 'Story': '3', 'Animation': '1...",https://myanimelist.net/reviews.php?id=286904
192110,287903,AgentK300,1281,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '3', 'Animation': '...",https://myanimelist.net/reviews.php?id=287903


In [6]:
animes = pd.read_csv("animes.csv")

In [7]:
animes = animes.drop_duplicates()

In [8]:
animes_dict = {}
for row,index in animes.iterrows():
    animes_dict[index["uid"]] = index["title"]

In [9]:
animes_dict

{28891: 'Haikyuu!! Second Season',
 23273: 'Shigatsu wa Kimi no Uso',
 34599: 'Made in Abyss',
 5114: 'Fullmetal Alchemist: Brotherhood',
 31758: 'Kizumonogatari III: Reiketsu-hen',
 37510: 'Mob Psycho 100 II',
 199: 'Sen to Chihiro no Kamikakushi',
 38000: 'Kimetsu no Yaiba',
 35247: 'Owarimonogatari 2nd Season',
 2904: 'Code Geass: Hangyaku no Lelouch R2',
 32935: 'Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou',
 34096: 'Gintama.',
 15335: 'Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare',
 918: 'Gintama',
 4181: 'Clannad: After Story',
 15417: "Gintama': Enchousen",
 30276: 'One Punch Man',
 16664: 'Kaguya-hime no Monogatari',
 4672: 'Koukaku Kidoutai 2.0',
 8038: 'Nodame Cantabile: Finale - Mine to Kiyora no Saikai',
 7588: 'Saraiya Goyou',
 1253: 'Saint Seiya: Meiou Hades Meikai-hen',
 20767: 'Noragami OVA',
 1425: 'Lupin III: Part II',
 35363: 'Kobayashi-san Chi no Maid Dragon: Valentine, Soshite Onsen! - Amari Kitai Shinaide Kudasai',
 22673: 'Kuroko no Basket

In [10]:
title = []
for uid in reviews["anime_uid"]:
    title.append(animes_dict[uid])
title_df = pd.DataFrame(title)
reviews["title"] = title_df

In [11]:
reviews = reviews.drop_duplicates()

In [12]:
reviews

,uid,profile,anime_uid,text,score,scores,link,title
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938,Gintama.
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117,Made in Abyss
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664,Haikyuu!! Second Season
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254,Code Geass: Hangyaku no Lelouch R2
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149,Clannad: After Story
...,...,...,...,...,...,...,...,...
182629,146535,iHitokage,2593,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=146535,Kara no Kyoukai 1: Fukan Fuukei
182631,13126,Onegai,2593,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '6', 'Animation': '9...",https://myanimelist.net/reviews.php?id=13126,Kara no Kyoukai 1: Fukan Fuukei
182633,127899,Murasa22,2593,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '9', 'Animation': '...",https://myanimelist.net/reviews.php?id=127899,Kara no Kyoukai 1: Fukan Fuukei
182636,286852,srry4apologizng,2593,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '5', 'Animation': '1...",https://myanimelist.net/reviews.php?id=286852,Kara no Kyoukai 1: Fukan Fuukei


In [13]:
reviews = reviews.drop(["profile","anime_uid","text","scores","link"], axis = 1)

In [14]:
reviews.rename(columns = {"uid" : "user_uid"}, inplace = True)

In [15]:
animes = animes.drop(["synopsis","genre","aired","episodes","members","popularity","ranked","img_url","link"],axis = 1)

In [16]:
animes.rename(columns = {"uid" : "anime_uid", "title" : "anime_title"})

,anime_uid,anime_title,score
0,28891,Haikyuu!! Second Season,8.82
1,23273,Shigatsu wa Kimi no Uso,8.83
2,34599,Made in Abyss,8.83
3,5114,Fullmetal Alchemist: Brotherhood,9.23
4,31758,Kizumonogatari III: Reiketsu-hen,8.83
...,...,...,...
19002,10075,Naruto x UT,7.50
19003,35828,Miira no Kaikata,7.50
19004,10378,Shinryaku!? Ika Musume,7.56
19005,33082,Kingsglaive: Final Fantasy XV,7.56


In [17]:
anime_titles = animes["title"]

In [18]:
'''def produce_matrix(reviews,anime_titles):
    matrix = pd.DataFrame()

    for anime_title in anime_titles:
        
        anime_title_column = []

        for row, index in reviews.iterrows():
            
            if index["title"] == anime_title:
                anime_title_column.append(index["score"])
            
            elif index["title"] != anime_title:
                anime_title_column.append(-1000)
        
        anime_title_column = pd.DataFrame(anime_title_column)
        anime_title_column.apply(lambda x: pd.NA if -1000 else x)
        pd.concat([matrix,anime_title_column], ignore_index = True)
    
    return matrix
'''

'def produce_matrix(reviews,anime_titles):\n    matrix = pd.DataFrame()\n\n    for anime_title in anime_titles:\n        \n        anime_title_column = []\n\n        for row, index in reviews.iterrows():\n            \n            if index["title"] == anime_title:\n                anime_title_column.append(index["score"])\n            \n            elif index["title"] != anime_title:\n                anime_title_column.append(-1000)\n        \n        anime_title_column = pd.DataFrame(anime_title_column)\n        anime_title_column.apply(lambda x: pd.NA if -1000 else x)\n        pd.concat([matrix,anime_title_column], ignore_index = True)\n    \n    return matrix\n'

In [19]:
def produce_matrix(reviews, anime_titles):
    pivot_table = reviews.pivot_table(index = "title", columns = "user_uid", values = "score", fill_value = pd.NA)
    return pivot_table

In [20]:
product_item_matrix = produce_matrix(reviews,anime_titles)

In [21]:
product_item_matrix

title,Haikyuu!! Second Season,Shigatsu wa Kimi no Uso,Made in Abyss,Fullmetal Alchemist: Brotherhood,Kizumonogatari III: Reiketsu-hen,Mob Psycho 100 II,Sen to Chihiro no Kamikakushi,Kimetsu no Yaiba,Owarimonogatari 2nd Season,Code Geass: Hangyaku no Lelouch R2,...,Yowamushi Pedal: Re:RIDE,Pokemon Movie 02: Maboroshi no Pokemon Lugia Bakutan,Mekakucity V's,Amagami SS,Net-juu no Susume Special,Naruto x UT,Miira no Kaikata,Shinryaku!? Ika Musume,Kingsglaive: Final Fantasy XV,Chuunibyou demo Koi ga Shitai!: Kirameki no... Slapstick Noel
user_uid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
325729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
325734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
model_knn = NearestNeighbors(n_neighbors = 20, algorithm = "brute", n_jobs = -1)
feature_matrix = csr_matrix(product_item_matrix)
model_knn.fit(feature_matrix)

In [19]:
#mean_user_ratings = np.mean(product_item_matrix, axis = 1).reshape(-1,1)

In [20]:
#product_item_matrix = product_item_matrix - mean_user_ratings 